# SUTRA 모델을 OpenAI 클라이언트와 함께 한국어 용도로 사용하기

<img src="https://play-lh.googleusercontent.com/_O9p4Z4yucA2NLmZBu9mTJCuBwXeT9NcbtrDN6I8gKlkIPRySV0adOmbyipjSj9Gew" width="150">

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1v2O_jaPIIi9Re891Y76NWCxNr4tJe3x9?usp=sharing)

## 소개

이 노트북에서는 Sutra 모델을 OpenAI 클라이언트와 함께 한국어 애플리케이션에 사용하는 방법을 설명합니다. Sutra는 인도 언어 처리 및 생성에 강점을 가지며, 한국어 작업에도 이상적인 선택입니다.

### 학습 내용

* Sutra 모델을 사용하기 위한 OpenAI 클라이언트 설정 방법  
* 한국어로 프롬프트를 생성하고 응답을 이해하는 방법  
* 다양한 한국어 사용 사례 탐색  
* 한국어 콘텐츠 생성을 위한 모범 사례


## API 키 받기

시작하기 전에 다음이 있는지 확인하세요:

1. SUTRA API 키 ([TWO AI의 SUTRA API 페이지](https://www.two.ai/sutra/api)에서 받으세요)
2. Python 및 Jupyter 노트북에 대한 기본적인 지식

이 노트북은 Google Colab에서 실행되도록 설계되었으므로 로컬 Python 설치가 필요하지 않습니다.

## 설정  
먼저, 필요한 라이브러리를 설치합니다.

In [ ]:
# 필수 패키지 설치
!pip install openai requests pandas matplotlib

### 필요한 라이브러리 불러오기


In [ ]:
# 필요한 라이브러리 불러오기
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI
from IPython.display import display, Markdown, HTML
from google.colab import userdata

## 인증  
Sutra API를 사용하려면 API 키를 설정해야 합니다.

In [ ]:
# 코드에서 사용하려면 예를 들어 다음과 같이 설정할 수 있습니다:
os.environ["SUTRA_API_KEY"] = userdata.get('SUTRA_API_KEY')

### Sutra API 엔드포인트로 OpenAI 클라이언트 생성


In [ ]:
# Sutra API 엔드포인트로 OpenAI 클라이언트 생성
client = OpenAI(
    base_url="https://api.two.ai/v2",
    api_key=os.environ.get("SUTRA_API_KEY")
)

## 도우미 함수

한국어 작업을 위한 Sutra API와 상호작용할 도우미 함수를 만듭니다.

In [ ]:
import time

def get_sutra_response(prompt, model="sutra-v2", temperature=0.7, max_tokens=1024):
    """
    주어진 프롬프트에 대해 Sutra에서 일반(비스트리밍) 응답을 받습니다.

    인수:
        prompt (str): 사용자 프롬프트 또는 질문
        model (str): 사용할 Sutra 모델
        temperature (float): 무작위성 제어 (0에서 1 사이)
        max_tokens (int): 생성할 최대 토큰 수

    반환:
        str: 모델의 응답
        float: 응답을 받는 데 걸린 시간
    """
    start_time = time.time()

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )

        end_time = time.time()
        result = response.choices[0].message.content
        time_taken = end_time - start_time

        # 결과 출력
        print("프롬프트:", prompt)
        print("응답:", result)
        print(f"소요 시간: {time_taken:.2f}초\n")

        return result, time_taken

    except Exception as e:
        end_time = time.time()
        print("오류:", str(e))
        return f"오류: {str(e)}", end_time - start_time


def get_sutra_streaming_response(prompt, model="sutra-v2", temperature=0.7, max_tokens=1024):
    """
    주어진 프롬프트에 대해 Sutra에서 스트리밍 응답을 받습니다.

    인수:
        prompt (str): 사용자 프롬프트 또는 질문
        model (str): 사용할 Sutra 모델
        temperature (float): 무작위성 제어 (0에서 1 사이)
        max_tokens (int): 생성할 최대 토큰 수

    반환:
        str: 전체 모델 응답
        float: 전체 응답 시간
    """
    start_time = time.time()

    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            stream=True
        )

        print("프롬프트:", prompt)
        print("응답:")

        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                full_response += content
                print(content, end="")

        print("\n")
        time_taken = time.time() - start_time
        print(f"소요 시간: {time_taken:.2f}초\n")

        return full_response, time_taken

    except Exception as e:
        end_time = time.time()
        print("오류:", str(e))
        return f"오류: {str(e)}", end_time - start_time


## 1. 기본 한국어 생성

Sutra를 사용하여 한국어 텍스트를 생성하는 기본 예제를 시작해 봅시다.


In [ ]:
# 간단한 한국어 텍스트 생성
korean_prompt = "안녕하세요, 제 이름은 Rahul입니다. 어떻게 지내세요?"

response, time_taken = get_sutra_response(korean_prompt)

프롬프트: 안녕하세요, 제 이름은 Rahul입니다. 어떻게 지내세요?
응답: 안녕하세요, Rahul님! 저는 잘 지내고 있습니다. 어떻게 도와드릴까요?
소요 시간: 2.50초



## 2. 한국어 정보 검색

Sutra를 사용하여 한국어로 정보를 검색해 봅시다.

In [ ]:
# 한국어 정보 검색
info_prompt = "인도의 5대 주요 축제에 대해 설명하고 그들의 중요성을 강조해주세요."

response, time_taken = get_sutra_response(info_prompt)

프롬프트: 인도의 5대 주요 축제에 대해 설명하고 그들의 중요성을 강조해주세요.
응답: 인도의 5대 주요 축제는 다음과 같습니다:

1. **디왈리 (Diwali)**: 
   - **설명**: 디왈리는 '빛의 축제'로, 힌두교에서 가장 중요한 축제 중 하나입니다. 일반적으로 10월 또는 11월에 기념되며, 라마 왕자가 악마 왕 라바나를 물리친 후 귀환하는 것을 기념합니다.
   - **중요성**: 디왈리는 악의 승리를 상징하며, 가족과 친구가 모여 서로에게 선물을 주고, 불꽃놀이와 전통 음식을 즐깁니다. 이 축제를 통해 사람들은 희망과 새로운 시작을 다짐합니다.

2. **홀리 (Holi)**: 
   - **설명**: 홀리는 봄의 도래를 축하하는 색의 축제로, 일반적으로 3월에 기념됩니다. 사람들은 서로에게 색색의 가루를 뿌리고 물놀이를 하며 즐깁니다.
   - **중요성**: 홀리는 사랑과 우정을 기념하며, 사회적 경계를 허물고 사람들 간의 연대를 강화하는 역할을 합니다. 이는 인도 문화에서 다양성의 중요성을 강조합니다.

3. **에크라트리 (Eid al-Fitr)**: 
   - **설명**: 에크라트리는 이슬람의 성스러운 금식 기간인 라마단이 끝난 후 기념되는 축제입니다. 이 날은 가족과 친구들이 모여 특별한 식사를 나누고, 기부를 통해 도움이 필요한 사람들을 지원합니다.
   - **중요성**: 에크라트리는 용서, 자선, 그리고 공동체의 중요성을 강조하며, 신앙의 힘을 재확인하는 기회입니다.

4. **재푸리 (Janmashtami)**: 
   - **설명**: 재푸리는 힌두교의 신 크리슈나의 탄생을 축하하는 축제입니다. 보통 8월에 기념되며, 밤새 경축 행사가 열립니다.
   - **중요성**: 이 축제는 신성함과 종교적 헌신을 강화하는 의미가 있으며, 많은 사람들이 특별한 기도를 드리고, 크리슈나의 삶을 이야기합니다.

5. **온암 (Onam)**: 
   - **설명**: 온암은 케랄라 주에서 기념되는 수확 축제로, 보통 8월 말에서 9월

## 3. 한국어 창작 글쓰기

Sutra는 한국어로 창의적인 콘텐츠를 생성할 수 있습니다.

In [ ]:
# 한국어 창작 글쓰기
creative_prompt = "히말라야에 관한 짧은 시를 써 주세요."

response, time_taken = get_sutra_response(creative_prompt, temperature=0.8)

프롬프트: 히말라야에 관한 짧은 시를 써 주세요.
응답: 히말라야의 위엄, 구름을 뚫고 솟아  
눈 덮인 봉우리, 신비로운 향기와 함께  
시간을 초월한 고요 속에 숨쉬며  
영혼의 여행자들을 기다리는 산.  

찬란한 태양이 그 위를 비출 때  
얼어붙은 세계에 따스함을 안겨  
끝없는 모험과 꿈의 땅,  
히말라야, 그대는 나의 영원한 사랑.
소요 시간: 2.63초



## 4. 한국어 요약

Sutra는 한국어 텍스트를 효과적으로 요약할 수 있습니다.

In [ ]:
# 한국어 요약 텍스트
long_text = """
인도의 독립 역사에는 길고 복잡한 과정이 있습니다. 1757년 플래시 전투에서 영국 동인도 회사가 벵골의 나와브 시라주드 다울라를 물리친 후, 영국은 인도를 지배하기 시작했습니다. 이후 회사는 점차 그 영향력을 확장했으며, 1858년에는 인도가 영국 왕실의 직접적인 지배 하에 들어갔습니다.

1857년의 반란, 즉 첫 번째 독립 전쟁은 영국 지배에 대한 첫 번째 주요 반란이었습니다. 비록 이 반란은 실패했지만, 인도인들 사이에서 민족 의식을 일깨웠습니다.

1885년에 인도 국민 회의가 결성되면서 독립 운동은 더 조직화되었습니다. 마하트마 간디의 지도 아래, 비폭력 저항과 사티아그라하를 통해 독립 요구가 제기되었습니다.

1942년 '인도 탈출 운동'이 시작되면서 영국은 인도를 떠나야 했습니다. 제2차 세계대전 이후 영국의 경제적 상태가 악화되었고, 결국 인도는 1947년 8월 15일 독립을 달성했으며, 인도와 파키스탄으로 나뉘었습니다.
"""

summarize_prompt = f"다음 텍스트를 100단어 이내로 요약하세요:\n\n{long_text}"

summary, time_taken = get_sutra_response(summarize_prompt, temperature=0.3)


프롬프트: 다음 텍스트를 100단어 이내로 요약하세요:


인도의 독립 역사에는 길고 복잡한 과정이 있습니다. 1757년 플래시 전투에서 영국 동인도 회사가 벵골의 나와브 시라주드 다울라를 물리친 후, 영국은 인도를 지배하기 시작했습니다. 이후 회사는 점차 그 영향력을 확장했으며, 1858년에는 인도가 영국 왕실의 직접적인 지배 하에 들어갔습니다.

1857년의 반란, 즉 첫 번째 독립 전쟁은 영국 지배에 대한 첫 번째 주요 반란이었습니다. 비록 이 반란은 실패했지만, 인도인들 사이에서 민족 의식을 일깨웠습니다.

1885년에 인도 국민 회의가 결성되면서 독립 운동은 더 조직화되었습니다. 마하트마 간디의 지도 아래, 비폭력 저항과 사티아그라하를 통해 독립 요구가 제기되었습니다.

1942년 '인도 탈출 운동'이 시작되면서 영국은 인도를 떠나야 했습니다. 제2차 세계대전 이후 영국의 경제적 상태가 악화되었고, 결국 인도는 1947년 8월 15일 독립을 달성했으며, 인도와 파키스탄으로 나뉘었습니다.

응답: 인도의 독립 역사는 1757년 영국 동인도 회사의 벵골 정복으로 시작되어, 1858년에는 영국 왕실의 직접 통치로 이어졌다. 1857년 첫 번째 독립 전쟁은 실패했지만 민족 의식을 고취시켰다. 1885년 인도 국민 회의가 결성되며 독립 운동이 조직화되었고, 마하트마 간디의 비폭력 저항이 중요한 역할을 했다. 1942년 '인도 탈출 운동'을 통해 영국은 인도를 떠나게 되었고, 1947년 8월 15일 인도는 독립을 이루며 파키스탄과 분리되었다.
소요 시간: 2.97초



## 5. 한국어 질문 응답

Sutra를 사용하여 한국어로 질문에 답해 봅시다.


In [ ]:
# 질문 응답을 위한 컨텍스트
context = """
인도 헌법은 세계에서 가장 긴 서면 헌법입니다. 1949년 11월 26일 채택되어 1950년 1월 26일에 시행되었습니다. Dr. B. R. Ambedkar는 인도 헌법의 주요 설계자로 여겨집니다. 원래의 헌법은 395개의 조항, 22개의 부분, 8개의 부록을 포함하고 있었습니다. 현재는 448개의 조항, 25개의 부분, 12개의 부록이 포함되어 있습니다. '사회주의', '세속주의', '통합'이라는 단어가 42차 수정 헌법(1976)에 의해 헌법 서문에 추가되었습니다.
"""

qa_prompt = f"제공된 정보를 바탕으로 다음 질문에 답하세요:\n\n{context}\n\n질문 1: 인도 헌법은 언제 시행되었나요?\n질문 2: 인도 헌법의 주요 설계자는 누구인가요?\n질문 3: 42차 수정 헌법에 의해 서문에 추가된 단어는 무엇인가요?"

answers, time_taken = get_sutra_response(qa_prompt, temperature=0.2)


프롬프트: 제공된 정보를 바탕으로 다음 질문에 답하세요:


인도 헌법은 세계에서 가장 긴 서면 헌법입니다. 1949년 11월 26일 채택되어 1950년 1월 26일에 시행되었습니다. Dr. B. R. Ambedkar는 인도 헌법의 주요 설계자로 여겨집니다. 원래의 헌법은 395개의 조항, 22개의 부분, 8개의 부록을 포함하고 있었습니다. 현재는 448개의 조항, 25개의 부분, 12개의 부록이 포함되어 있습니다. '사회주의', '세속주의', '통합'이라는 단어가 42차 수정 헌법(1976)에 의해 헌법 서문에 추가되었습니다.


질문 1: 인도 헌법은 언제 시행되었나요?
질문 2: 인도 헌법의 주요 설계자는 누구인가요?
질문 3: 42차 수정 헌법에 의해 서문에 추가된 단어는 무엇인가요?
응답: 질문 1: 인도 헌법은 1950년 1월 26일에 시행되었습니다.

질문 2: 인도 헌법의 주요 설계자는 Dr. B. R. Ambedkar입니다.

질문 3: 42차 수정 헌법(1976)에 의해 서문에 추가된 단어는 '사회주의', '세속주의', '통합'입니다.
소요 시간: 2.41초



## 6. 한국어-힌디어 번역

Sutra는 한국어와 힌디어 간의 번역을 효과적으로 할 수 있습니다.


In [ ]:
# 영어에서 한국어로 번역
english_text = "India is a diverse country where many languages, religions, and cultures coexist. Its rich traditions and history give it a unique place in the world."

translate_en_to_ko_prompt = f"다음 영어 텍스트를 한국어로 번역하세요:\n\n{english_text}"

translation_ko, time_taken = get_sutra_response(translate_en_to_ko_prompt)

프롬프트: 다음 영어 텍스트를 한국어로 번역하세요:

India is a diverse country where many languages, religions, and cultures coexist. Its rich traditions and history give it a unique place in the world.
응답: 인도는 여러 언어, 종교, 문화가 공존하는 다양한 나라입니다. 그 풍부한 전통과 역사는 세계에서 독특한 위치를 부여합니다.
소요 시간: 4.30초



In [ ]:
# 영어에서 한국어로 번역
english_text = "Artificial intelligence is transforming the way we live and work. It has applications in healthcare, education, transportation, and many other fields. As technology advances, it's important to ensure that AI is developed ethically and responsibly."

translate_en_to_ko_prompt = f"다음 영어 텍스트를 한국어로 번역하세요:\n\n{english_text}"

translation_ko, time_taken = get_sutra_response(translate_en_to_ko_prompt)

프롬프트: 다음 영어 텍스트를 한국어로 번역하세요:

Artificial intelligence is transforming the way we live and work. It has applications in healthcare, education, transportation, and many other fields. As technology advances, it's important to ensure that AI is developed ethically and responsibly.
응답: 인공지능은 우리의 생활과 업무 방식을 변화시키고 있습니다. 이것은 의료, 교육, 운송 및 여러 다른 분야에서 응용되고 있습니다. 기술이 발전함에 따라 AI가 윤리적이고 책임감 있게 개발되도록 하는 것이 중요합니다.
소요 시간: 2.54초



## 7. 한국어 감정 분석

Sutra를 사용하여 한국어 텍스트에 대한 감정 분석을 수행해 보겠습니다.

In [ ]:
# 한국어 리뷰 감정 분석
reviews = [
    "이 영화는 정말 좋았습니다. 이야기가 몰입감 있었고 연기가 훌륭했습니다.",
    "음식은 전혀 맛이 없었고 서비스도 매우 형편없었습니다.",
    "제품은 괜찮지만 가격이 조금 비쌉니다.",
    "이 책이 마음에 들었고, 저자가 주제를 아주 잘 설명했습니다.",
    "여행은 실망스러웠습니다. 호텔은 더럽고 직원들이 불친절했습니다."
]

sentiment_prompt = "다음 한국어 리뷰에 대해 감정 분석을 수행하고 각 리뷰를 '긍정적', '부정적', 또는 '중립적'으로 분류하세요:\n\n"

for i, review in enumerate(reviews):
    sentiment_prompt += f"{i+1}. {review}\n"

sentiment_analysis, time_taken = get_sutra_response(sentiment_prompt, temperature=0.1)

프롬프트: 다음 한국어 리뷰에 대해 감정 분석을 수행하고 각 리뷰를 '긍정적', '부정적', 또는 '중립적'으로 분류하세요:

1. 이 영화는 정말 좋았습니다. 이야기가 몰입감 있었고 연기가 훌륭했습니다.
2. 음식은 전혀 맛이 없었고 서비스도 매우 형편없었습니다.
3. 제품은 괜찮지만 가격이 조금 비쌉니다.
4. 이 책이 마음에 들었고, 저자가 주제를 아주 잘 설명했습니다.
5. 여행은 실망스러웠습니다. 호텔은 더럽고 직원들이 불친절했습니다.

응답: 1. 긍정적
2. 부정적
3. 중립적
4. 긍정적
5. 부정적
소요 시간: 2.22초



## 8. 한국어 콘텐츠 생성 다양한 형식

Sutra는 다양한 형식으로 한국어 콘텐츠를 생성할 수 있습니다.


In [ ]:
# 한국어 블로그 게시물 생성
blog_prompt = "주제 '인도 요리와 건강에 미치는 영향'에 대한 블로그 게시물을 작성하세요. 제목, 소개, 주요 사항, 결론을 포함하세요."

blog_post = get_sutra_streaming_response(blog_prompt, temperature=0.7)

프롬프트: 주제 '인도 요리와 건강에 미치는 영향'에 대한 블로그 게시물을 작성하세요. 제목, 소개, 주요 사항, 결론을 포함하세요.
응답:
### 제목: 인도 요리와 건강에 미치는 영향

#### 소개
인도 요리는 그 풍부한 맛과 다양한 재료로 세계적으로 유명합니다. 그러나 인도 요리는 단순히 맛있는 음식 그 이상입니다. 다양한 향신료, 신선한 채소, 그리고 지역 특산물들이 어우러져 건강에도 긍정적인 영향을 미칠 수 있습니다. 이 블로그에서는 인도 요리의 주요 재료와 그들이 우리 몸에 미치는 건강적 이점에 대해 알아보겠습니다.

#### 주요 사항

1. **향신료의 건강 이점**
   - **강황**: 강황은 항염 및 항산화 효과가 뛰어난 커큐민을 포함하고 있어 면역 체계를 강화하고 염증을 줄이는 데 도움을 줍니다.
   - **고수**: 고수는 소화를 돕고 해독 작용을 하며, 비타민 A와 C가 풍부하여 피부 건강에도 기여합니다.
   - **생강**: 생강은 소화 불량, 메스꺼움, 염증을 완화하는 데 효과적이며, 면역력을 높이는 데 도움을 줍니다.

2. **신선한 재료 사용**
   - 인도 요리는 신선한 채소, 과일, 콩류, 곡물 등을 널리 사용합니다. 이러한 재료들은 섬유질과 영양소가 풍부하여 소화 건강을 개선하고 만성 질환 예방에 기여합니다.

3. **균형 잡힌 식단**
   - 전통적인 인도 식사는 탄수화물, 단백질, 지방이 균형 있게 포함되어 있습니다. 예를 들어, 쌀이나 난(인도식 빵)과 함께 제공되는 다양한 채소 카레는 영양소를 고르게 섭취할 수 있는 좋은 방법입니다.

4. **채식 중심의 식사**
   - 인도는 세계에서 가장 많은 채식 인구를 보유한 국가 중 하나입니다. 채식 식단은 심장병, 비만, 당뇨병 등의 위험을 줄이는 데 도움이 됩니다.

5. **소금과 설탕의 조절**
   - 전통적으로 인도 요리는 소금과 설탕의 사용을 절제하여 건강을 고려한 조리법을 따릅니다. 이는 고혈압이나 당뇨병 환자에게 긍정적인 영향을 미칠 수 있습

## 9. 결론

이 노트북에서는 **한국어 언어 응용 프로그램**을 위한 Sutra 모델과 OpenAI 클라이언트를 사용하는 방법을 탐구했습니다. 우리는 다양한 사용 사례를 다뤘습니다:

* 기본 한국어 텍스트 생성
* 한국어 정보 검색
* 한국어 창작 글쓰기
* 한국어 텍스트 요약
* 한국어 질문 응답
* 한국어-힌디어 번역
* 한국어 감정 분석
* 다양한 형식의 한국어 콘텐츠 생성
* 한국어 코드 전환
* 한국어 문화적 맥락 이해
* 한국어로 된 구조화된 출력 생성

Sutra는 한국어 및 기타 인도 언어에서 강력한 능력을 발휘하므로 한국어 사용자를 대상으로 하는 응용 프로그램이나 한국어 처리가 필요한 응용 프로그램에 적합한 선택입니다.


## 10. 추가 리소스

**한국어** 언어와 함께 Sutra를 사용하는 방법에 대해 더 알고 싶다면 아래 리소스를 참조하세요:

1. [Sutra API 문서](https://docs.two.ai)
2. [Sutra 모델 변형](https://docs.two.ai/models)
3. [다국어 기능에 대해 더 알아보기](https://docs.two.ai/multilingual)

여러분은 Sutra를 사용하여 **한국어** 및 기타 인도 언어로 응용 프로그램을 구축하는 것을 권장합니다!
